In [ ]:
!pip install transformers datasets peft torch

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig
from datasets import load_dataset

In [ ]:
dataset = load_dataset("imdb")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

train_dataset = dataset["train"].map(tokenize_function, batched=True)
test_dataset = dataset["test"].map(tokenize_function, batched=True)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Configure LoRA
lora_config = LoraConfig(
    r=8,  # Low-rank factor
    lora_alpha=16,  # Scaling factor for LoRA
    target_modules=["encoder.layer.*.attention.self.query", "encoder.layer.*.attention.self.key"],  # Target attention layers
    lora_dropout=0.1,  # Dropout in LoRA layers
)

# Apply LoRA to the model
peft_model = get_peft_model(model, lora_config)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

peft_model.save_pretrained("./peft_model")
